In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import cv2
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.image import flip_left_right, flip_up_down
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score

from IPython.display import FileLink

In [3]:
train_data = pd.read_csv("/kaggle/input/planets-dataset/planet/planet/train_classes.csv", dtype = str)
train_data.head()

In [4]:
tag_labels = train_data['tags'].map(lambda x: x.split(" "))
split_labels = tag_labels.values
labels = list(set([y for x in split_labels for y in x]))

In [5]:
labels

In [6]:
def load_data(df_train, labels, target_size):
  
    X_train = []
    y_train = []

    label_map = {l: i for i, l in enumerate(labels)}
    inv_label_map = {i: l for l, i in label_map.items()}

    for f, tags in df_train.values:
        img = cv2.imread('/kaggle/input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(f))
        targets = np.zeros(17)
        for t in tags.split(' '):
            targets[label_map[t]] = 1 

        X_train.append(cv2.resize(img, target_size, interpolation = cv2.INTER_CUBIC))
        y_train.append(targets)
        
    y_train = np.array(y_train, np.uint8)
    X_train = np.array(X_train, np.float32) / 255.

    return X_train, y_train

In [7]:
X, Y = load_data(train_data, labels, target_size = (32, 32))

In [8]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2)

In [9]:
vgg_19 = VGG19(include_top = False, weights = 'imagenet', input_shape = (32, 32, 3))
vgg_19.trainable = False

In [10]:
tl_output = vgg_19.output
next_layer = Flatten()(tl_output)
dense1 = Dense(units = 1012, activation = 'relu')(next_layer)
dropout = Dropout(0.5)(dense1)
dense2 = Dense(units = 506, activation = 'relu')(dropout)
output = Dense(units=17, activation='sigmoid')(dense2)
model = Model(inputs = vgg_19.input, outputs = output)

In [11]:
model.summary()

In [12]:
optim = SGD(learning_rate=0.001, momentum = 0.9, nesterov = True)
model.compile(optimizer = optim, metrics = ['accuracy'], loss = 'binary_crossentropy')

In [13]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('accuracy') >= 0.95):
      print('\nReached 95% accuracy so cancelling training!')
      self.model.stop_training = True

myCallback_object = myCallback()

In [14]:
reduceLr = ReduceLROnPlateau(monitor='accuracy', factor=0.1, patience=5, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

In [15]:
model.fit(x = X_train, y = Y_train, validation_data = (X_val, Y_val), epochs = 50, callbacks = [myCallback_object, reduceLr])

In [16]:
test_data = pd.read_csv('/kaggle/input/planets-dataset/planet/planet/sample_submission.csv', dtype = str)
test_data.head()

In [39]:
os.chdir('/kaggle/working')

In [17]:
# kaggle submission
import csv
X_tests = []
submission = []
for file in os.listdir('/kaggle/input/planets-dataset/planet/planet/test-jpg'):
    filename = file.split('.')[0]
    
    img = cv2.imread('/kaggle/input/planets-dataset/planet/planet/test-jpg/{}.jpg'.format(filename))
    img = cv2.resize(img, (32, 32), cv2.INTER_CUBIC)
    
    X_tests.append(img)
    
    submission.append(filename)

for files in os.listdir("/kaggle/input/planets-dataset/test-jpg-additional/test-jpg-additional"):
    filenames = files.split('.')[0]
    img_2 = (cv2.imread('/kaggle/input/planets-dataset/test-jpg-additional/test-jpg-additional/{}.jpg'.format(filenames)))
    img_2 = cv2.resize(img_2, (32, 32), cv2.INTER_CUBIC)
    
    X_tests.append(img_2)
    
    submission.append(filenames)
    
targets = np.zeros(17)
      
X_tests = np.array(X_tests, np.float32) / 255

X_preds = model.predict(X_tests, batch_size=128)

with open('understanding_the_amazon_from_space.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(('image_name', 'tags'))
    for i, image in enumerate(submission):
        csv_writer.writerow((image, ' '.join(np.array(labels)[X_preds[i] > 0.2])))

In [18]:
FileLink(r'understanding_the_amazon_from_space.csv')